# Support tickets classification model

This notebook aims to classify support tickets using Bert transfromer model. 

By Constant Fotie Moghommahie

[fotiecodes](https://fotiecodes.com)


### 1. Import and install neccessary libs


In [48]:
!pip3 install nltk
!pip3 install --upgrade tensorflow



[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip


In [49]:
import tensorflow as tf
#import tensorflow_hub as hub
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import re
import unicodedata
from nltk.corpus import stopwords
from tensorflow import keras
from tensorflow.keras.layers import Dense,Dropout, Input
#from tqdm import tqdm
import pickle
from sklearn.metrics import confusion_matrix,f1_score,classification_report
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras import regularizers

In [50]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fotiem.constant/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [51]:
from transformers import BertTokenizer, TFBertModel, BertConfig,TFDistilBertModel,DistilBertTokenizer,DistilBertConfig

### 2.  Preprocessing and cleaning functions

In [52]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    return " ".join(clean_words) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

### 3. Load and Inspect our dataset
We will first load and and preview our dataset. So we can check for any missing values.

In [53]:
# Load and inspect the new dataset
data_file = './data/all_tickets.csv'

# Read the data from the CSV file into a pandas DataFrame
data=pd.read_csv(data_file,encoding='ISO-8859-1')

# Display the first few rows of the DataFrame
data.head()

,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
0,NaN,hi since recruiter lead permission approve req...,1,4,2,21,71,3,4
1,connection with icon,icon dear please setup icon per icon engineers...,1,6,22,7,26,3,4
2,work experience user,work experience user hi work experience studen...,1,5,13,7,32,3,4
3,requesting for meeting,requesting meeting hi please help follow equip...,1,5,13,7,32,3,4
4,reset passwords for external accounts,re expire days hi ask help update passwords co...,1,4,2,76,4,3,4


>  Removing Unnamed Columns, dropping NaN data and resetting the index after dropping some rows/columns containing NaN dataset and finally shuffling the dataset

In [54]:
data = data.loc[:, ~data.columns.str.contains('Unnamed: 2', case=False)] 
data = data.loc[:, ~data.columns.str.contains('Unnamed: 3', case=False)] 
data = data.loc[:, ~data.columns.str.contains('Unnamed: 4', case=False)] 
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))
data=data.dropna()
data=data.reset_index(drop=True)
print('File has {} rows and {} columns'.format(data.shape[0],data.shape[1]))
data = shuffle(data)



data.head()

File has 48549 rows and 9 columns
File has 47837 rows and 9 columns


,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
16842,po,tuesday hi please log installation install rec...,1,5,13,7,32,3,4
44125,survey gizmo performance issue,sent monday february status investigating app ...,0,4,21,77,91,1,2
18405,create customer users groups for,hello please create customers users locations,1,4,3,7,4,3,4
4727,request for stand for a monitor,stand monitor hello please provide stand monit...,1,5,13,7,32,3,4
1825,new starter,re starter form attached thursday december pm ...,1,4,2,94,40,3,4


###  4. Loading DistilBERT Tokenizer and the DistilBERT model

In [55]:
dbert_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [56]:
dbert_model = TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['activation_13', 'vocab_layer_norm', 'vocab_transform', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


###  5. Preparing input for the model

In [57]:
# Set the maximum length of the input sentences
max_len = 32

# Get the sentences and labels from the data
sentences = data['body']
labels = data['category']

# Print the length of the sentences and labels
len(sentences), len(labels)


(47837, 47837)

####  Let's take a sentence from the dataset and understand the input and output of the DistilBERT

In [58]:
sentences[0]

'icon dear please setup icon per icon engineers please let other details needed thanks lead'

In [59]:
dbert_tokenizer.tokenize(sentences[0])

['icon',
 'dear',
 'please',
 'setup',
 'icon',
 'per',
 'icon',
 'engineers',
 'please',
 'let',
 'other',
 'details',
 'needed',
 'thanks',
 'lead']

> Input ids and the attention masks from the tokenizer

In [60]:
dbert_inp=dbert_tokenizer.encode_plus(sentences[0],add_special_tokens = True,max_length =20,pad_to_max_length = True,truncation=True)
dbert_inp

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


{'input_ids': [101, 12696, 6203, 3531, 16437, 12696, 2566, 12696, 6145, 3531, 2292, 2060, 4751, 2734, 4283, 2599, 102, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]}

In [61]:
dbert_inp['input_ids']

[101,
 12696,
 6203,
 3531,
 16437,
 12696,
 2566,
 12696,
 6145,
 3531,
 2292,
 2060,
 4751,
 2734,
 4283,
 2599,
 102,
 0,
 0,
 0]

>  DistilBERT model output: Give input_ids and the attention_mask obtained from the tokenizer. The output will be a tuple of the size (1,max_len,768)

In [62]:
id_inp=np.asarray(dbert_inp['input_ids'])
mask_inp=np.asarray(dbert_inp['attention_mask'])
out=dbert_model([id_inp.reshape(1,-1),mask_inp.reshape(1,-1)])
type(out),out

(transformers.modeling_tf_outputs.TFBaseModelOutput,
 TFBaseModelOutput(last_hidden_state=<tf.Tensor: shape=(1, 20, 768), dtype=float32, numpy=
 array([[[-0.14008602, -0.1231253 ,  0.07263236, ..., -0.3267806 ,
           0.04675016,  0.43017605],
         [-0.36188334,  0.2787817 ,  0.5043748 , ..., -0.03066764,
          -0.10666229,  0.11820538],
         [ 0.2077359 ,  0.47382352,  0.70303   , ..., -0.03583608,
          -0.3576398 , -0.10728897],
         ...,
         [-0.00958552,  0.10920835,  0.25056088, ..., -0.00907475,
          -0.3153267 ,  0.17255719],
         [-0.00315598,  0.11595375,  0.25825262, ..., -0.01347051,
          -0.32174903,  0.15192199],
         [-0.02049228,  0.11264408,  0.26872396, ..., -0.05816982,
          -0.28254673,  0.11775325]]], dtype=float32)>, hidden_states=None, attentions=None))

> Obtain the embeddings of a sentence from the output

In [63]:
out[0][:,0,:]

<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-1.40086025e-01, -1.23125300e-01,  7.26323575e-02,
        -8.36244375e-02, -3.50452848e-02, -2.44505659e-01,
         3.50453258e-01,  4.78630960e-01, -3.12227197e-02,
        -2.45661944e-01, -1.27533868e-01,  4.65072207e-02,
         2.69581527e-02,  1.26936018e-01, -1.20387040e-01,
         1.19356841e-01,  5.23970798e-02,  3.97658587e-01,
         3.53917554e-02,  5.12354970e-02,  3.25017571e-02,
        -4.14746106e-01, -8.00804049e-02, -2.90628940e-01,
        -9.52901840e-02, -1.12366155e-01, -1.20869190e-01,
        -1.73668742e-01, -2.39612758e-01,  7.61435414e-03,
        -3.68524119e-02,  1.19285576e-01, -7.38689899e-02,
        -3.43415797e-01, -5.97161576e-02, -5.51657304e-02,
         1.64421290e-01,  2.88035303e-01,  1.42615229e-01,
         7.63547570e-02, -1.18344821e-01, -2.24739477e-01,
         4.11063612e-01, -1.91477984e-02,  3.57589304e-01,
         9.15082358e-03, -2.36848235e+00, -1.05126031e-01,
      

> Decode the original sentence from the tokenizer 

In [64]:
dbert_tokenizer.decode(dbert_inp['input_ids'])

'[CLS] icon dear please setup icon per icon engineers please let other details needed thanks lead [SEP] [PAD] [PAD] [PAD]'

In [65]:
print('Available labels: ',data.category.unique())
num_classes = len(data.category.unique())
data.head()


Available labels:  [ 5  4  6  7 11  8 12  9  3 10  1  0  2]


,title,body,ticket_type,category,sub_category1,sub_category2,business_service,urgency,impact
16842,po,tuesday hi please log installation install rec...,1,5,13,7,32,3,4
44125,survey gizmo performance issue,sent monday february status investigating app ...,0,4,21,77,91,1,2
18405,create customer users groups for,hello please create customers users locations,1,4,3,7,4,3,4
4727,request for stand for a monitor,stand monitor hello please provide stand monit...,1,5,13,7,32,3,4
1825,new starter,re starter form attached thursday december pm ...,1,4,2,94,40,3,4


###  6. Create a basic NN model using DistilBERT embeddings to get the predictions

In [66]:
def create_model():
    inps = Input(shape = (max_len,), dtype='int64')
    masks= Input(shape = (max_len,), dtype='int64')
    dbert_layer = dbert_model(inps, attention_mask=masks)[0][:,0,:]
    dense = Dense(512,activation='relu',kernel_regularizer=regularizers.l2(0.01))(dbert_layer)
    dropout= Dropout(0.5)(dense)
    pred = Dense(num_classes, activation='softmax',kernel_regularizer=regularizers.l2(0.01))(dropout)
    model = tf.keras.Model(inputs=[inps,masks], outputs=pred)
    print(model.summary())
    return model   

> Feel free to add more Dense and Dropout layers with variable units and the regularizers

In [67]:
model=create_model()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 tf_distil_bert_model_1 (TF  TFBaseModelOutput(last_hid   6636288   ['input_3[0][0]',             
 DistilBertModel)            den_state=(None, 32, 768),   0          'input_4[0][0]']             
                              hidden_states=None, atten                                           
                             tions=None)                                                      

> Prepare the model input

In [68]:
input_ids=[]
attention_masks=[]

for sent in sentences:
    dbert_inps=dbert_tokenizer.encode_plus(sent,add_special_tokens = True,max_length =max_len,pad_to_max_length = True,return_attention_mask = True,truncation=True)
    input_ids.append(dbert_inps['input_ids'])
    attention_masks.append(dbert_inps['attention_mask'])

input_ids=np.asarray(input_ids)
attention_masks=np.array(attention_masks)
labels=np.array(labels)

/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2285: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [75]:
len(input_ids),len(attention_masks),len(labels)

(47837, 47837, 47837)

> Save the model input in the pickle files to use it later without performing the above steps

In [76]:
print('Preparing the pickle file.....')

pickle_inp_path='./data/pickle_files/dbert_inp.pkl'
pickle_mask_path='./data/pickle_files/dbert_mask.pkl'
pickle_label_path='./data/pickle_files/dbert_label.pkl'

Preparing the pickle file.....


In [78]:
pickle.dump((input_ids),open(pickle_inp_path,'wb'))
pickle.dump((attention_masks),open(pickle_mask_path,'wb'))
pickle.dump((labels),open(pickle_label_path,'wb'))


print('Pickle files saved as ',pickle_inp_path,pickle_mask_path,pickle_label_path)

Pickle files saved as  ./data/pickle_files/dbert_inp.pkl ./data/pickle_files/dbert_mask.pkl ./data/pickle_files/dbert_label.pkl


In [79]:
print('Loading the saved pickle files..')

input_ids=pickle.load(open(pickle_inp_path, 'rb'))
attention_masks=pickle.load(open(pickle_mask_path, 'rb'))
labels=pickle.load(open(pickle_label_path, 'rb'))

print('Input shape {} Attention mask shape {} Input label shape {}'.format(input_ids.shape,attention_masks.shape,labels.shape))

Loading the saved pickle files..
Input shape (47837, 32) Attention mask shape (47837, 32) Input label shape (47837,)


In [80]:
label_class_dict = dict(enumerate(data['category'].unique()))
target_names = label_class_dict.values()

> Train Test split and setting up the loss function, accuracy and optimizer for the model. 

In [81]:
train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)

print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))


log_dir='dbert_model'
model_save_path='./dbert_model.h5'

callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

Train inp shape (38269, 32) Val input shape (9568, 32)
Train label shape (38269,) Val label shape (9568,)
Train attention mask shape (38269, 32) Val attention mask shape (9568, 32)


In [82]:
callbacks= [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,save_weights_only=True,monitor='val_loss',mode='min',save_best_only=True),keras.callbacks.TensorBoard(log_dir=log_dir)]
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

### Training

In [83]:
history=model.fit([train_inp,train_mask],train_label,batch_size=16,epochs=5,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Epoch 1/5


/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/keras/src/backend.py:5727: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


2392/2392 [==============================] - 9941s 4s/step - loss: 4.1571 - accuracy: 0.8225 - val_loss: 2.2996 - val_accuracy: 0.8486
Epoch 2/5
2392/2392 [==============================] - 8870s 4s/step - loss: 1.5658 - accuracy: 0.8659 - val_loss: 1.1435 - val_accuracy: 0.8412
Epoch 3/5
2392/2392 [==============================] - 8996s 4s/step - loss: 0.7965 - accuracy: 0.8950 - val_loss: 0.7721 - val_accuracy: 0.8464
Epoch 4/5
2392/2392 [==============================] - 12610s 5s/step - loss: 0.4778 - accuracy: 0.9218 - val_loss: 0.6512 - val_accuracy: 0.8504
Epoch 5/5
2392/2392 [==============================] - 10476s 4s/step - loss: 0.3179 - accuracy: 0.9426 - val_loss: 0.6385 - val_accuracy: 0.8446


### Tensorboard visualization (Training-Testing curve)

In [88]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [89]:
%tensorboard --logdir {log_dir}

Reusing TensorBoard on port 6006 (pid 28401), started 0:17:36 ago. (Use '!kill 28401' to kill it.)

### We will Increase the number of epochs in order to decrease the loss further
Here we will use the saved model for predictions and calculating the evaluation metrics

In [86]:
trained_model = create_model()
trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 input_6 (InputLayer)        [(None, 32)]                 0         []                            
                                                                                                  
 tf_distil_bert_model_1 (TF  TFBaseModelOutput(last_hid   6636288   ['input_5[0][0]',             
 DistilBertModel)            den_state=(None, 32, 768),   0          'input_6[0][0]']             
                              hidden_states=None, atten                                           
                             tions=None)                                                    

In [114]:
preds = trained_model.predict([val_inp,val_mask],batch_size=16)
pred_labels = preds.argmax(axis=1)
f1 = f1_score(val_label, pred_labels, average='weighted')
f1

598/598 [==============================] - 960s 2s/step


0.8381137426950556

In [129]:
print(target_names)
# print(target_names.shape)
print(val_label.shape)
print(pred_labels.shape)


# we print F1 score and classification report
print('F1 score:', f1)
print('Classification Report:')
print(classification_report(val_label, pred_labels))

print('Training and saving built model...')

[ 5  4  6  7 11  8 12  9  3 10  1  0  2]
(9568,)
(9568,)
F1 score: 0.8381137426950556
Classification Report:
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        16
           3       0.55      0.23      0.32        26
           4       0.90      0.92      0.91      6787
           5       0.72      0.79      0.76      1858
           6       0.56      0.39      0.46       502
           7       0.54      0.34      0.42       185
           8       0.90      0.72      0.80        39
           9       0.80      0.32      0.46        37
          11       0.72      0.54      0.62       112
          12       0.00      0.00      0.00         6

    accuracy                           0.84      9568
   macro avg       0.57      0.43      0.47      9568
weighted avg       0.84      0.84      0.84      9568

Training and saving built model...


/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/fotiem.constant/.pyenv/versions/3.10.6/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(av